\[I wrote the following in 2006.\]

# Don't go postal — it's just a unique word puzzle

Are you familiar with those word morphing puzzles in which you start with one word, change one letter at a time yet making a valid word each time, and end up at another word? Well, I've got something like that, but it's something you've never seen before.

Start with a four-letter word. At each step, change either the first two letters or the last two letters to make a different word. Here's the catch: **For every word, the first two letters and last two letters must each be a valid USPS two-letter state code.** Think of it as playing a game of Twister on a map of the United States. The first two letters of your word represent your left leg, and the last two letters represent your right leg. For example, if you start with your left leg in Colorado and your right leg in Nebraska, you've spelled CONE. From there you can move your left leg to Wisconsin (WINE), but not to Vermont (VTNE?). Or you can move your right leg to Oklahoma (COOK), but not to Texas (COTX?). Let's say that you moved from CONE to WINE. Note that from here you cannot move your left leg to Arkansas, attempting to spell NEAR, because according to the rules you'd really be spelling ARNE (which is not allowed because, as Cubs fans know, it's a proper name).  Also, you're allowed to have both legs in the same state, as long as you can fit them (you may be glad that RIRI isn't a word).

Here are some easy ones to get you started:

- Go from COOK to MEAL.
- Go from MAMA to PAPA.
- Go from MIME to LAME. \[Heh heh.\]

A little more complex:

- Go from COIL to WIND.
- Go from PAIN to SCAR.
- Go from ARCO to CAMO.

Now the tricky stuff:

- Find a word from which you can't move.
- Without backtracking, go through Hawaii on your way from CANE to LAVA.

And the coup de grace:

- Starting anywhere you want, create as dirty of a story as you can on your way to LAID, being sure to pass through Oregon and Rhode Island.

---

Back in the present (2026). How many four-letter state abbreviation strings are even valid words?

I downloaded a couple of word lists from <a href="https://github.com/dolph/dictionary/tree/master">Dolph Mathews's repo</a>.

In [1]:
states = [s.lower() for s in "AK AL AR AZ CA CO CT DC DE FL GA HI IA ID IL IN KS KY LA MA MD ME MI MN MO MS MT NC ND NE NH NJ NM NV NY OH OK OR PA RI SC SD TN TX UT VA VT WA WI WV WY".split()]
assert len(states) == 51

In [2]:
def pvs(set_):
    L = sorted(set_)
    print(f"{len(L)} values: {L[0]}, ..., {L[-1]}")

In [3]:
state_strings = set([s1 + s2 for s1 in states for s2 in states])
pvs(state_strings)

2601 values: akak, ..., wywy


In [4]:
def read_list(file):
    with open(file) as f:
        return set(word
                   for word in f.read().strip().split('\n')
                   if len(word) == 4)

In [5]:
enable = read_list('enable1.txt')
pvs(enable)

3903 values: aahs, ..., zyme


In [6]:
popular = read_list('popular.txt')
pvs(popular)

1997 values: abba, ..., zoom


In [7]:
len(state_strings & enable)

137

In [8]:
len(state_strings & popular)

72

That's not so many words. I'll build an initial version of a graph:

In [9]:
from collections import defaultdict

class Graph:
    def __init__(self, words, nodes=state_strings):
        self.words = words
        self.nodes = sorted(node for node in nodes if node in words)
        self.adj = defaultdict(list)
        for i, n1 in enumerate(self.nodes[:-1]):
            for n2 in self.nodes[i+1:]:
                if n1 == n2:
                    continue
                if n1[:2] == n2[:2] or n1[2:] == n2[2:]:
                    self.adj[n1].append(n2)
                    self.adj[n2].append(n1)

In [10]:
ge = Graph(enable)

In [11]:
ge.nodes[:8]

['akin', 'alar', 'alga', 'alky', 'alma', 'alme', 'alms', 'arak']

This list may include more than I'm really looking for.

In [12]:
[node for node in ge.nodes if node not in ge.adj]

[]

And it has no isolated words.

In [13]:
gp = Graph(popular)

In [14]:
gp.nodes[:8]

['alma', 'aria', 'arid', 'arms', 'cain', 'came', 'cams', 'cane']

In [15]:
[node for node in gp.nodes if node not in gp.adj]

['flak']

Well, I'm pretty sure FLAK is what I had in mind for the word from which you can't move.

In the enable list:

In [16]:
ge.adj['flak']

['arak', 'kyak']

Given that all other words can reach each other, let's find the shortest paths.

In [17]:
class Graph:
    def __init__(self, words, nodes=state_strings):
        self.words = words
        self.nodes = sorted(node for node in nodes if node in words)
        self.adj = defaultdict(list)
        for i, n1 in enumerate(self.nodes[:-1]):
            for n2 in self.nodes[i+1:]:
                if n1 == n2:
                    continue
                if n1[:2] == n2[:2] or n1[2:] == n2[2:]:
                    self.adj[n1].append(n2)
                    self.adj[n2].append(n1)

    def shortest_paths(self, node1, node2):
        current = [(node1, )]
        visited = set()
        visited.add(node1)
        pending_visited = set()
        pending = []
        results = []
        while current and not results:
            for path in current:
                for next_node in self.adj[path[-1]]:
                    if next_node == node2:
                        results.append(path + (next_node, ))
                        continue
                    if next_node in visited:
                        continue
                    pending.append(path + (next_node, ))
                    pending_visited.add(next_node)
            current = pending[:]
            pending.clear()
            visited.update(pending_visited)
            pending_visited.clear()
        return results

In [18]:
gp = Graph(popular)

In [19]:
gp.shortest_paths('cook', 'meal')

[('cook', 'coal', 'meal')]

In [20]:
gp.shortest_paths('mama', 'papa')

[('mama', 'maid', 'paid', 'papa'),
 ('mama', 'mail', 'pail', 'papa'),
 ('mama', 'main', 'pain', 'papa')]

In [21]:
gp.shortest_paths('mime', 'lame')

[('mime', 'lame')]

In [22]:
gp.shortest_paths('coil', 'wind')

[('coil', 'code', 'wide', 'wind'),
 ('coil', 'cone', 'wine', 'wind'),
 ('coil', 'wail', 'wand', 'wind')]

In [23]:
gp.shortest_paths('pain', 'scar')

[('pain', 'coin', 'coal', 'deal', 'dear', 'scar'),
 ('pain', 'coin', 'coco', 'deco', 'dear', 'scar'),
 ('pain', 'main', 'many', 'deny', 'dear', 'scar')]

In [24]:
gp.shortest_paths('arco', 'camo')

[]

Hmm.

In [25]:
ge = Graph(enable)

In [26]:
ge.shortest_paths('arco', 'camo')

[]

Double hmm.

In [27]:
'arco' in ge.nodes, 'camo' in ge.nodes

(True, False)

That won't do...

In [28]:
gx = Graph(enable | {'camo'})

In [29]:
gx.shortest_paths('arco', 'camo')

[('arco', 'arid', 'caid', 'camo'),
 ('arco', 'arms', 'cams', 'camo'),
 ('arco', 'deco', 'demo', 'camo')]

Not sure which of those last two paths I had in mind.

Now let's see about CANE to LAVA...

In [30]:
cane_to_hi = [gp.shortest_paths('cane', node) for node in gp.nodes if 'hi' in node]
hi_to_lava = [gp.shortest_paths(node, 'lava') for node in gp.nodes if 'hi' in node]

In [31]:
from itertools import chain

for path1 in chain.from_iterable(cane_to_hi):
    for path2 in chain.from_iterable(hi_to_lava):
        if path1[-1] != path2[0]:
            continue
        if not set(path1) & set(path2[1:]):
            print(path1 + path2[1:])

('cane', 'cone', 'code', 'hide', 'hind', 'land', 'lava')
('cane', 'wine', 'wide', 'hide', 'hind', 'land', 'lava')
('cane', 'mine', 'mind', 'hind', 'land', 'lava')
('cane', 'wine', 'wind', 'hind', 'land', 'lava')
